In [3]:
import pandas as pd
import requests
from io import StringIO

station_id = "USC00427061"  # Provo Airport
start_year = 2020
end_year = 2024

all_years = []

for year in range(start_year, end_year + 1):
    print(f"Downloading {year} weather data...")

    url = (
        "https://www.ncei.noaa.gov/access/services/data/v1"
        f"?dataset=daily-summaries&stations={station_id}"
        f"&startDate={year}-01-01&endDate={year}-12-31"
        "&units=standard&format=csv"
)

    # SSL verify disabled to avoid campus/Mac certificate issues
    response = requests.get(url, verify=False)

    # Convert CSV text → DataFrame
    df_year = pd.read_csv(StringIO(response.text))
    all_years.append(df_year)

# Combine all years
df = pd.concat(all_years, ignore_index=True)

# Select snowfall + snow depth + precipitation
df = df[['DATE', 'SNOW', 'SNWD', 'PRCP']]

# Convert dates
df['DATE'] = pd.to_datetime(df['DATE'])

print(df.head())
print(df.tail())
print(df.describe())

/Users/ellawalker/Documents/STAT 386/Project/weather_api_project/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncei.noaa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


/Users/ellawalker/Documents/STAT 386/Project/weather_api_project/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncei.noaa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


/Users/ellawalker/Documents/STAT 386/Project/weather_api_project/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncei.noaa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


/Users/ellawalker/Documents/STAT 386/Project/weather_api_project/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncei.noaa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


/Users/ellawalker/Documents/STAT 386/Project/weather_api_project/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ncei.noaa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


        DATE  SNOW  SNWD  PRCP
0 2024-05-17   NaN   NaN  0.00
1 2024-05-18   NaN   NaN  0.00
2 2024-05-19   NaN   NaN  0.01
3 2024-05-20   NaN   NaN  0.13
4 2024-05-21   NaN   NaN  0.00
          DATE  SNOW  SNWD  PRCP
216 2024-12-27   NaN   NaN  0.21
217 2024-12-28   NaN   NaN  0.22
218 2024-12-29   NaN   NaN  0.01
219 2024-12-30   NaN   NaN  0.00
220 2024-12-31   NaN   NaN  0.00
                                DATE  SNOW  SNWD        PRCP
count                            221   0.0   0.0  221.000000
mean   2024-09-09 13:27:57.828054272   NaN   NaN    0.016199
min              2024-05-17 00:00:00   NaN   NaN    0.000000
25%              2024-07-17 00:00:00   NaN   NaN    0.000000
50%              2024-09-11 00:00:00   NaN   NaN    0.000000
75%              2024-11-05 00:00:00   NaN   NaN    0.000000
max              2024-12-31 00:00:00   NaN   NaN    0.420000
std                              NaN   NaN   NaN    0.056833


/var/folders/f4/ftsj1j7n5837rlz4_q0rks500000gn/T/ipykernel_19113/2475976317.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(all_years, ignore_index=True)
